In [1]:
import sys
import tkinter
import pandas as pd
import numpy as np
import tkinter.messagebox
from tkintermapview import TkinterMapView
from pyswip import Prolog

In [2]:
df = pd.read_csv('./Destinations.csv')
print(df.shape)
df.head()

(102, 13)


,Destinations,country,region,Climate,Budget,Activity,Demographics,Duration,Cuisine,History,Natural Wonder,Accommodation,Language
0,Tokyo,Japan,East Asia,Temperate,High,Cultural,Solo,Long,Asian,Modern,Mountains,Luxury,Japanese
1,Ottawa,Canada,North America,Cold,Medium,Adventure,Family-friendly,Medium,European,Modern,Forests,Mid-range,English
2,Mexico City,Mexico,North America,Temperate,Low,Cultural,Senior,Short,Latin American,Ancient,Mountains,Budget,Spanish
3,Rome,Italy,Southern Europe,Temperate,High,Cultural,Solo,Medium,European,Ancient,Beaches,Luxury,Italian
4,Brasilia,Brazil,South America,Tropical,Low,Adventure,Family-friendly,Long,Latin American,Modern,Beaches,Budget,Portuguese


In [3]:
# TODO 1: read destinations' descriptions from Destinations.csv and add them to the prolog knowledge base
################################################################################################
# STEP1: Define the knowledge base of illnesses and their symptoms

def empty_kb(prolog):
    prolog.retractall("my_destination(_,_)")
    prolog.retractall("country(_,_)")
    prolog.retractall("region(_,_)")
    prolog.retractall("climate(_,_)")
    prolog.retractall("budget(_,_)")
    prolog.retractall("activity(_,_)")
    prolog.retractall("demographic(_,_)")
    prolog.retractall("duration(_,_)")
    prolog.retractall("cuisine(_,_)")
    prolog.retractall("history(_,_)")
    prolog.retractall("natural_wonder(_,_)")
    prolog.retractall("accommodation(_,_)")
    prolog.retractall("language(_,_)")


prolog = Prolog()
empty_kb(prolog)

for i in df.index:
    dest = df['Destinations'][i].replace("'", "`")
    prolog.assertz(f"my_destination('{dest}')")
    prolog.assertz(f"country('{df['country'][i]}')")
    prolog.assertz(f"region('{df['region'][i]}')")
    prolog.assertz(f"climate('{df['Climate'][i]}')")
    prolog.assertz(f"budget('{df['Budget'][i]}')")
    prolog.assertz(f"activity('{df['Activity'][i]}')")
    prolog.assertz(f"demographic('{df['Demographics'][i]}')")
    prolog.assertz(f"duration('{df['Duration'][i]}')")
    prolog.assertz(f"cuisine('{df['Cuisine'][i]}')")
    prolog.assertz(f"history('{df['History'][i]}')")
    prolog.assertz(f"natural_wonder('{df['Natural Wonder'][i]}')")
    prolog.assertz(f"accommodation('{df['Accommodation'][i]}')")
    prolog.assertz(f"language('{df['Language'][i]}')")



# TODO 2: extract unique features from the Destinations.csv and save them in a dictionary
################################################################################################

key_words = dict() 
for col in ['Destinations', 'country', 'region', 'Climate', 'Budget', 'Activity', 'Demographics', 'Duration', 'Cuisine', 'History', 'Natural Wonder', 'Accommodation', 'Language']:
    key_words[col] = df[col].unique().tolist()

In [4]:


class App(tkinter.Tk):

    APP_NAME = "map_view_demo.py"
    WIDTH = 800
    HEIGHT = 750  # This is now the initial size, not fixed.

    def __init__(self, *args, **kwargs):
        tkinter.Tk.__init__(self, *args, **kwargs)

        self.title(self.APP_NAME)
        self.geometry(f"{self.WIDTH}x{self.HEIGHT}")

        # Configure the grid
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)  # Text area and submit button combined row
        self.grid_rowconfigure(1, weight=4)  # Map row

        # Upper part: Text Area and Submit Button
        self.text_area = tkinter.Text(self, height=5)  # Reduced height for text area
        self.text_area.grid(row=0, column=0, pady=(10, 0), padx=10, sticky="nsew")

        self.submit_button = tkinter.Button(self, text="Submit", command=self.process_text)
        self.submit_button.grid(row=0, column=0, pady=(0, 10), padx=10, sticky="se")  # Placed within the same cell as text area

        # Lower part: Map Widget
        self.map_widget = TkinterMapView(self)
        self.map_widget.grid(row=1, column=0, sticky="nsew")

        self.marker_list = []  # Keeping track of markers
        self.marker_path = None


    def __init__(self, *args, **kwargs):
        tkinter.Tk.__init__(self, *args, **kwargs)

        self.title(self.APP_NAME)
        self.geometry(f"{self.WIDTH}x{self.HEIGHT}")

        # Configure the grid
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)  # Text area can expand/contract.
        self.grid_rowconfigure(1, weight=0)  # Submit button row; doesn't need to expand.
        self.grid_rowconfigure(2, weight=3)  # Map gets the most space.

        # Upper part: Text Area and Submit Button
        self.text_area = tkinter.Text(self)
        self.text_area.grid(row=0, column=0, pady=10, padx=10, sticky="nsew")
        
        self.submit_button = tkinter.Button(self, text="Submit", command=self.process_text)
        self.submit_button.grid(row=1, column=0, pady=10, sticky="ew")

        # Lower part: Map Widget
        self.map_widget = TkinterMapView(self)
        self.map_widget.grid(row=2, column=0, sticky="nsew")

        self.marker_list = []  # Keeping track of markers

    def check_connections(self, results):
        print('result2 ', results)
        locations = []
        for result in results:
            city  = result["City"]
            locations.append(city)
            # TODO 5: create the knowledgebase of the city and its connected destinations using Adjacency_matrix.csv

            prolog = Prolog()

            prolog.retractall("directly_connected(_,_)")
            prolog.retractall("connected(_,_)")

                    
            df = pd.read_csv('./Adjacency_matrix.csv')
            mx = df.to_numpy()

            for row in mx:
                for col in row[1:]:
                    if col == 1:
                        prolog.assertz(f"directly_connected({row[0]}, {mx[col-1][0]})")

            prolog.assertz("connected(X, Y) :- directly_connected(X, Y)")
            prolog.assertz("connected(X, Y) :- directly_connected(Y, X)")

            prolog.assertz("connected(X, Y) :- directly_connected(X, Z), connected(Z, Y)")

        return locations

    def process_text(self):
        """Extract locations from the text area and mark them on the map."""
        text = self.text_area.get("1.0", "end-1c")  # Get text from text area
        results = self.extract_locations(text)  # Extract locations (you may use a more complex method here)


        # TODO 4: create the query based on the extracted features of user desciption 
        ################################################################################################
        print(results)
        locations = self.check_connections(results)


        # TODO 6: if the number of destinations is less than 6 mark and connect them
        ################################################################################################
        print(locations)
        locations = ['mexico_city','rome' ,'brasilia']
        self.mark_locations(locations)

    def mark_locations(self, locations):
        """Mark extracted locations on the map."""
        for address in locations:
            marker = self.map_widget.set_address(address, marker=True)
            if marker:
                self.marker_list.append(marker)
        self.connect_marker()
        self.map_widget.set_zoom(1)  # Adjust as necessary, 1 is usually the most zoomed out


    def connect_marker(self):
        print(self.marker_list)
        position_list = []

        for marker in self.marker_list:
            position_list.append(marker.position)

        if hasattr(self, 'marker_path') and self.marker_path is not None:
            self.map_widget.delete(self.marker_path)

        if len(position_list) > 0:
            self.marker_path = self.map_widget.set_path(position_list)

    def extract_locations(self, text):
        """Extract locations from text. A placeholder for more complex logic."""
        # Placeholder: Assuming each line in the text contains a single location name
        # TODO 3: extract key features from user's description of destinations
        ################################################################################################
        results = list()
        for word in text.split(' '):
            for key in ['Destinations', 'Climate', 'region', 'Climate', 'Budget', 'Activity', 'Demographics', 'Duration', 'Cuisine', 'History', 'Natural Wonder', 'Accommodation', 'Language']:
                if word in key_words[key]:
                    query = key + f"(City, {word})"
                    result = list(prolog.query(query))
                    results.append(result)

        final_results = []
        if results:
            final_results = results[0]
            for lst in results[1:]:
                final_results = list(set(final_results) & set(lst))

        # return [line.strip() for line in text.split('\n') if line.strip()]
        return final_results

    def start(self):
        self.mainloop()




In [5]:
# Don't run for now :)
if __name__ == "__main__":
    app = App()
    app.start()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/tmp/ipykernel_75865/2415367278.py", line 96, in process_text
    results = list(prolog.query(query))
  File "/home/pouya/dev/ai/basics/project4/fol-darcodezan/venv/lib/python3.10/site-packages/pyswip/prolog.py", line 126, in __call__
    raise PrologError("".join(["Caused by: '", query, "'. ",
pyswip.prolog.PrologError: Caused by: 'destination(City,_, _, _, low, _, _, _, _, _, _, _, _)'. Returned: 'error(existence_error(procedure, /(destination, 13)), context(/(pyrun, 2), _4782))'.


In [6]:
print(key_words)

{'Destinations': ['Tokyo', 'Ottawa', 'Mexico City', 'Rome', 'Brasilia', 'Canberra', 'New Delhi', 'Pretoria', 'Madrid', 'Moscow', 'Berlin', 'Bangkok', 'Stockholm', 'Cairo', 'Wellington', 'Ankara', 'Buenos Aires', 'Kathmandu', 'Athens', 'Nairobi', 'Lisbon', 'Oslo', 'Jakarta', 'Hanoi', 'Dublin', 'Beijing', 'Paris', 'Santiago', 'Rabat', 'Singapore', 'Lima', 'Helsinki', 'Prague', 'Manila', 'Brussels', 'Kuala Lumpur', 'Amsterdam', 'Washington D.C.', 'London', 'Seoul', 'Bogota', 'Warsaw', 'Jerusalem', 'Kyiv', 'Vienna', 'Copenhagen', 'Dhaka', 'Montevideo', 'Bern', 'Islamabad', 'Budapest', 'Zagreb', 'Colombo', 'Tehran', 'Bratislava', 'Cancun', 'Venice', 'Florence', 'Milan', 'Saint Petersburg', 'Vladivostok', 'New York City', 'Los Angeles', 'Chicago', 'Miami', 'Shiraz', 'Isfahan', 'Mashhad', 'Tabriz', 'Yazd', 'Shanghai', 'Guangzhou', 'Chengdu', "Xi'an", 'Bergen', 'Tromsø', 'Marseille', 'Lyon', 'Sentosa', 'Marina Bay', 'Cusco', 'Arequipa', 'Busan', 'Incheon', 'Medellin', 'Cali', 'Lahore', 'Karach